# This notebook demonstrates how to execute async/sync workflows with Radical and Dask execution backends.

### Sync workflows with Dask
We will execute 5 simple workflows in sync approach (sequentially) using Dask Distributed execution backend.

In [ ]:
%env FLOW_JUPYTER_ASYNC=FALSE
import time

from radical.flow import WorkflowEngine
from radical.flow import DaskExecutionBackend

backend = DaskExecutionBackend({'n_workers': 2,
                               'threads_per_worker': 1})

flow = WorkflowEngine(backend=backend)

@flow.executable_task
def task1(*args):
    return '/bin/date'

@flow.executable_task
def task2(*args):
    return '/bin/date'

@flow.executable_task
def task3(*args):
    return '/bin/date'

def run_wf(wf_id):

    print(f'Starting workflow {wf_id} at {time.time()}')
    t3 = task3(task1(), task2())
    print(t3.result())
    print (f'Workflow {wf_id} completed at {time.time()}\n')

for i in range(5):
    run_wf(i)

backend.shutdown()

env: FLOW_JUPYTER_ASYNC=FALSE
Starting workflow 0 at 1744309456.756387
Thu Apr 10 06:24:17 PM UTC 2025

Workflow 0 completed at 1744309457.3185828

Starting workflow 1 at 1744309457.318625
Thu Apr 10 06:24:18 PM UTC 2025

Workflow 1 completed at 1744309458.297442

Starting workflow 2 at 1744309458.2974577
Thu Apr 10 06:24:19 PM UTC 2025

Workflow 2 completed at 1744309459.3037918

Starting workflow 3 at 1744309459.303816
Thu Apr 10 06:24:20 PM UTC 2025

Workflow 3 completed at 1744309460.3250942

Starting workflow 4 at 1744309460.3251545
Thu Apr 10 06:24:21 PM UTC 2025

Workflow 4 completed at 1744309461.327662

Shutdown is triggered, terminating the resources gracefully


### Async workflows with RadicalPilot 
We will execute 5 workflows in async approach (in prallel) using execution backend.
Note that, you can use any other backends with the same approach like: ThreadPool, PorcessPool, or DaskParallel

In [ ]:
%env FLOW_JUPYTER_ASYNC=TRUE
import time
import asyncio

from radical.flow import WorkflowEngine
from radical.flow import RadicalExecutionBackend


async def main():
    backend = RadicalExecutionBackend({'resource': 'local.localhost'})
    flow = WorkflowEngine(backend=backend)
    
    @flow.executable_task
    async def task1(*args):
        return '/bin/echo "I got executed at" && /bin/date'
    
    @flow.executable_task
    async def task2(*args):
        return '/bin/echo "I got executed at" && /bin/date'
    
    @flow.executable_task
    async def task3(*args):
        return '/bin/echo "I got executed at" && /bin/date'
    
    @flow.executable_task
    async def task4(*args):
        return '/bin/echo "I got executed at" && /bin/date'
    
    @flow.executable_task
    async def task5(*args):
        return '/bin/echo "I got executed at" && /bin/date'
    
    async def run_wf(wf_id):
        print(f'Starting workflow {wf_id} at {time.time()}')
        t1 = task1()
        t2 = task2(t1)
        t3 = task3(t1, t2)
        t4 = task4(t3)

        res = await t4
        if res:
            print(f'task4 from {wf_id} got result {res}')
            t5 = task5()
            print('submitted task5')
            await t5

        print (f'Workflow {wf_id} completed at {time.time()}\n')

    # Run workflows concurrently
    results = await asyncio.gather(*[run_wf(i) for i in range(2)])

    backend.shutdown()

await main()

env: FLOW_JUPYTER_ASYNC=TRUE
Resource backend started successfully

Starting workflow 0 at 1744309643.3847566
Starting workflow 1 at 1744309643.385239
task4 from 1 got result I got executed at
Thu Apr 10 06:27:38 PM UTC 2025

submitted task5
task4 from 0 got result I got executed at
Thu Apr 10 06:27:38 PM UTC 2025

submitted task5
Workflow 0 completed at 1744309659.005562

Workflow 1 completed at 1744309659.005703

Shutdown is triggered, terminating the resources gracefully
